# Job 2

Il job 2 valuta, annualmente, per ogni stato e categoria di business la valutazione media per ogni fascia di prezzo, assegnando un giudizio.

Nel dettaglio:

- per ogni business viene calcolata la valutazione media delle recensioni raggruppandole per anno;
- aggregando in base alla categoria di business, lo stato e la fascia di prezzo viene calcolata la media delle valutazioni medie delle recensioni;
- in base alla valutazione media viene elaborato un attributo aggiuntivo "business suggestion" che fornisce un giudizio sulle categorie di business, come segue:
  - valutazione media < 2: "Not recommended"
  - valutazione media 2–3.5: "Discreet"
  - valutazione media 3.5–4.5: "Recommended"
  - valutazione media > 4.5: "Highly recommended"



### Schema definitions

In [21]:
import org.apache.spark

import org.apache.spark


In [22]:
for ((k,v) <- sc.getPersistentRDDs) {
  v.unpersist()
}

In [23]:
import org.apache.spark.sql.types._
import java.sql.Timestamp

val reviewSchema = StructType(
  Seq(
    StructField("user_id",  StringType,            nullable = true),
    StructField("name",     StringType,            nullable = true),
    StructField("time",     LongType,              nullable = false),
    StructField("rating",   DoubleType,            nullable = true),
    StructField("text",     StringType,            nullable = true),
    StructField("pics",     ArrayType(StringType), nullable = true),
    StructField("resp",     StructType(
      Seq(
        StructField("time", LongType,              nullable = false),
        StructField("text", StringType,            nullable = true)
      )
    ),                                             nullable = true),
    StructField("gmap_id",  StringType,            nullable = false),
  )
)

case class Response(time: Timestamp, text: Option[String])

case class Review(
  user_id: Option[String],
  name: Option[String],
  time: Timestamp,
  rating: Option[Double],
  text: Option[String],
  pics: Seq[String],
  resp: Option[Response],
  gmap_id: String
)

import org.apache.spark.sql.types._
import java.sql.Timestamp
reviewSchema: org.apache.spark.sql.types.StructType = StructType(StructField(user_id,StringType,true),StructField(name,StringType,true),StructField(time,LongType,false),StructField(rating,DoubleType,true),StructField(text,StringType,true),StructField(pics,ArrayType(StringType,true),true),StructField(resp,StructType(StructField(time,LongType,false),StructField(text,StringType,true)),true),StructField(gmap_id,StringType,false))
defined class Response
defined class Review


In [24]:
val metadataSchema = StructType(
  Seq(
    StructField("name",             StringType,                                 nullable = true),
    StructField("address",          StringType,                                 nullable = true),
    StructField("gmap_id",          StringType,                                 nullable = false),
    StructField("description",      StringType,                                 nullable = true),
    StructField("latitude",         DoubleType,                                 nullable = false),
    StructField("longitude",        DoubleType,                                 nullable = false),
    StructField("category",         ArrayType(StringType),                      nullable = true),
    StructField("avg_rating",       DoubleType,                                 nullable = false),
    StructField("num_of_reviews",   IntegerType,                                nullable = false),
    StructField("price",            StringType,                                 nullable = true),
    StructField("hours",            ArrayType(ArrayType(StringType)),           nullable = true),
    StructField("MISC",             MapType(StringType, ArrayType(StringType)), nullable = true),
    StructField("state",            StringType,                                 nullable = true),
    StructField("relative_results", ArrayType(StringType),                      nullable = true),
    StructField("url",              StringType,                                 nullable = false),
  )
)

case class Metadata(
  name: Option[String],
  address: Option[String],
  gmap_id: String,
  description: Option[String],
  latitude: Double,
  longitude: Double,
  category: Seq[String],
  avg_rating: Double,
  num_of_reviews: Int,
  price: Option[String],
  hours: Seq[Seq[String]],
  MISC: Map[String, Seq[String]],
  state: Option[String],
  relative_results: Seq[String],
  url: String
)

metadataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(name,StringType,true),StructField(address,StringType,true),StructField(gmap_id,StringType,false),StructField(description,StringType,true),StructField(latitude,DoubleType,false),StructField(longitude,DoubleType,false),StructField(category,ArrayType(StringType,true),true),StructField(avg_rating,DoubleType,false),StructField(num_of_reviews,IntegerType,false),StructField(price,StringType,true),StructField(hours,ArrayType(ArrayType(StringType,true),true),true),StructField(MISC,MapType(StringType,ArrayType(StringType,true),true),true),StructField(state,StringType,true),StructField(relative_results,ArrayType(StringType,true),true),StructField(url,StringType,false))
defined class Metadata


### Dataset load and parse

In [ ]:
import java.nio.file.Paths

val projectDir: String = Paths.get(System.getProperty("user.dir")).getParent.getParent.getParent.toString
val reviewsPath = s"$projectDir/dataset/sample-reviews.ndjson"
val metadataPath = s"$projectDir/dataset/metadata.ndjson"

import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("NDJSON Reader")
  .master("local[*]") // Needed in local mode
  .getOrCreate()

val reviewsDf = spark.read
  .schema(reviewSchema)
  .json(reviewsPath)
  .withColumn("pics", when (col("pics") isNull, array()) otherwise col("pics"))
  .withColumn("time", from_unixtime(col("time") / 1000).cast("timestamp"))
  .withColumn("resp", 
    when (
      col("resp") isNotNull, 
      struct(
        from_unixtime(col("resp.time") / 1000).cast("timestamp").alias("time"),
        col("resp.text").cast(StringType).alias("text")
      )
    ) otherwise lit(null)
  )
  .as[Review]

val metadataDf = spark.read
  .schema(metadataSchema)
  .json(metadataPath)
  .withColumn("category", when (col("category") isNull, array()) otherwise col("category"))
  .withColumn("hours", when (col("hours") isNull, array()) otherwise col("hours"))
  .withColumn("relative_results", when (col("relative_results") isNull, array()) otherwise col("relative_results"))
  .withColumn("MISC",
    when (
      col("MISC") isNotNull,
      col("MISC").cast(MapType(StringType, ArrayType(StringType)))
    ) otherwise typedLit(Map.empty[String, Seq[String]])
  )
  .as[Metadata]

reviewsDf.printSchema()
metadataDf.printSchema()

// Unforturnately, it seems that Spark does not support case classes in RDDs. It throws ArrayStoreException
// when trying to collect the RDD... [see also [here](https://github.com/adtech-labs/spylon-kernel/issues/40)]
val reviewsRdd = reviewsDf.rdd
  .map(Review.unapply)
  .map(_.get)
  .map { case review @ (_, _, _, _, _, _, resp, _) => review.copy(_7 = resp.map(Response.unapply(_).get)) }
val metaRdd = metadataDf.rdd.map(Metadata.unapply).map(_.get)

root
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- pics: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- resp: struct (nullable = true)
 |    |-- time: timestamp (nullable = true)
 |    |-- text: string (nullable = true)
 |-- gmap_id: string (nullable = true)

root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- gmap_id: string (nullable = true)
 |-- description: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- avg_rating: double (nullable = true)
 |-- num_of_reviews: integer (nullable = true)
 |-- price: string (nullable = true)
 |-- hours: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: 

import java.nio.file.Paths
projectDir: String = /Users/teo/Universita/Magistrale/BIG_DATA/bd-project25
reviewsPath: String = /Users/teo/Universita/Magistrale/BIG_DATA/bd-project25/dataset/sample-reviews.ndjson
metadataPath: String = /Users/teo/Universita/Magistrale/BIG_DATA/bd-project25/dataset/metadata.ndjson
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@68c6c9f
reviewsDf: org.apache.spark.sql.Dataset[Review] = [user_id: string, name: string ... 6 more fields]
metadataDf: org.apache.spark.sql.Dataset[Metadata] = [name: string, address: string ... 13 more fields]
reviewsRdd: org.apache.spark.rdd.RDD[(Option[String], Option[String], java.sql.Timestamp, Option[Double], Option[String], Seq[String], Option[(java.sql.Tim...


In [32]:
/** This regex captures the state abbreviation between a comma and the ZIP code. */
private val StateRegex = """,\s*([A-Z]{2})\s+\d{5}""".r

/** The map of states that are considered for the analysis. */
val consideredStates = Map(
  "Alabama" -> "AL",
  "Mississippi" -> "MS",
  "New Hampshire" -> "NH",
  "New Mexico" -> "NM",
  "Washington" -> "WA",
)

/** Regex to match state names in the address. */
val StateNameRegex = s"""\\b(${consideredStates.keys.mkString("|")})\\b""".r

/** Regex to match state abbreviations in the address. */
val StateAbbrevRegex = s"""\\b(${consideredStates.values.mkString("|")})\\b""".r

/**
 * Extracts the state from the given address.
 * @param address
 *   the optional address string
 * @return
 *   the state abbreviation or "Unknown" if no valid state is found
 * @see
 *   [[consideredStates]]
 */
def toState(address: Option[String]): String = address
  .flatMap { addr =>
    StateRegex
      .findFirstMatchIn(addr)
      .map(_.group(1))
      .orElse(StateNameRegex.findFirstMatchIn(addr).map(stateName => consideredStates(stateName.group(1))))
      .orElse(StateAbbrevRegex.findFirstMatchIn(addr).map(_.group(1)))
  }
  .filter(consideredStates.values.toSeq.contains)
  .getOrElse("Unknown")

consideredStates: scala.collection.immutable.Map[String,String] = Map(New Mexico -> NM, Washington -> WA, Alabama -> AL, Mississippi -> MS, New Hampshire -> NH)
StateNameRegex: scala.util.matching.Regex = \b(New Mexico|Washington|Alabama|Mississippi|New Hampshire)\b
StateAbbrevRegex: scala.util.matching.Regex = \b(NM|WA|AL|MS|NH)\b
toState: (address: Option[String])String


In [ ]:
/** This function converts price string into a new one with custom symbol */
def withSymbol(price: Option[String], symbol: String): Option[String] =
  price.map(s => List.fill(s.length)(symbol).mkString)

withSymbol: (price: Option[String], symbol: String)Option[String]


### Job2 - Computation

---

**Metadata**: (name, address, <ins>gmap_id</ins>, description, latitude, longitude, category, avg_rating, num_of_reviews, price, hours, misc, state, relative_results, url)

**Review**: (user_id, name, time, rating, text, pics, responses, <ins>gmap_id</ins>)

---

In [43]:
// avg rating mapper to description
def ratingToSuggestion(rating: Double): String =
  rating match {
    case r if r <= 2.0 => "Not recommended"
    case r if r > 2.0 && r <= 3.5 => "Discreet"
    case r if r > 3.5 && r <= 4.5 => "Recommended"
    case r if r > 4.5 => "Highly recommended"
    case _ => "Undefined"
  }

ratingToSuggestion: (rating: Double)String


In [ ]:
val metadata = metaRdd
  .map{ case (_, address, gmap_id, _, _, _, category, _, _, price, _, _, _, _, _) => 
    gmap_id -> (category, toState(address), withSymbol(price, "*"))
  }

val reviewsData = reviewsRdd
  .map { case (_, _, time, rating, _, _, _, gmap_id) => 
    gmap_id -> (time.toLocalDateTime.getYear, rating)
  }

val joinData = reviewsData
  .join(metadata)
  .filter(_._2._1._2.isDefined)
  .filter(_._2._2._3.isDefined)
  .map{ case (gmap_id, ((year, rating), (category, address, price))) => 
    gmap_id -> (year, rating.get, category, address, price)
  }


metadata: org.apache.spark.rdd.RDD[(String, (Seq[String], String, Option[String]))] = MapPartitionsRDD[25] at map at <console>:37
reviewsData: org.apache.spark.rdd.RDD[(String, (Int, Option[Double]))] = MapPartitionsRDD[26] at map at <console>:42
joinData: org.apache.spark.rdd.RDD[(String, (Int, Double, Seq[String], String, Option[String]))] = MapPartitionsRDD[32] at map at <console>:50


In [45]:

val results = joinData
  .map{case (gmap_id, (year, rating, categories, state, price)) =>
    (gmap_id, year, categories, state, price) -> rating
  }
  .groupByKey()
  .flatMap{ case ((gmap_id, year, categories, state, price), rate) =>
    categories.map(category => 
      (category, state, price, year) -> rate.sum/rate.size
    )
  }
  .groupByKey()
  .map{ case (key, avgRates) =>
    val avgRate = avgRates.sum/avgRates.size
    key -> (avgRate, ratingToSuggestion(avgRate))
  }  


results: org.apache.spark.rdd.RDD[((String, String, Option[String], Int), (Double, String))] = MapPartitionsRDD[37] at map at <console>:43


### Save output result

In [46]:
val outputDirPath = s"$projectDir/output"
val outputPath = s"$outputDirPath/job2-output"

outputDirPath: String = /Users/teo/Universita/Magistrale/BIG_DATA/bd-project25/output
outputPath: String = /Users/teo/Universita/Magistrale/BIG_DATA/bd-project25/output/job2-output


In [47]:
import org.apache.spark.sql.SaveMode
results
  .map { case ((category, state, price, year), (avgRate, suggestion)) =>
        (category, state, price, year, avgRate, suggestion)
  }
  .coalesce(1)
  .toDF("category", "state", "price", "year", "avg_rating", "business suggestion")
  .write
  .format("csv")
  .option("header", "true")
  .mode(SaveMode.Overwrite)
  .save(s"file://$outputPath")

import org.apache.spark.sql.SaveMode
